In [1]:
import os
os.chdir("../")

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    data_path: Path
    tokenizer_path: Path
    updated